Data Loaders etc

In [3]:
# TODO: Need to find a solution with package management
import sys
sys.path.insert(1, 'C:/Users/morit/OneDrive/UNI/Master/WS22/APP-RAS/Programming/data_pipeline')
from data_sampler import WeightedSampler

In [4]:
from data_sampler import WeightedSampler
from dataset import CARLADataset, CARLADatasetMultiProcessing
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision
import numpy as np
import torch
import torch.nn as nn
import os

In [34]:
path_ege_data = os.path.join("..", "data", "Dataset Ege")
"""
config = {"used_inputs": ["rgb", "depth", "measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake"],
        "seq_len": 1
        }
"""
config = {"used_inputs": ["rgb","measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake"],
        "seq_len": 1
        }

train_dataset = CARLADataset(root_dir=path_ege_data, config=config, transform=None)
weighted_sampler = WeightedSampler(dataset=train_dataset)
print(train_dataset.__len__())

Town10HD_Scenario10_route9_11_28_17_59_39 has varying number of data files among input folders. It got discarded from the dataset.
2005


In [18]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=16, sampler=weighted_sampler, collate_fn=None)

In [19]:
batch = next(iter(train_dataloader))

In [20]:
batch.keys()

dict_keys(['idx', 'rgb', 'speed', 'steer', 'throttle', 'brake'])

Model ResNet

https://www.pluralsight.com/guides/introduction-to-resnet

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import time
import copy
import os

batch_size = 16
learning_rate = 1e-3

transforms = transforms.Compose(
[
    transforms.ToTensor()
])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def imshow(inp, title=None):
    
    inp = inp.cpu() if device else inp
    inp = inp.numpy().transpose((1, 2, 0))
    
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)
    


In [52]:
images = next(iter(train_dataloader))["rgb"] 
images = images.transpose(1,4)
images = images[:,:,:,:,0]
images.shape
print("images-size:", images.shape)

images-size: torch.Size([16, 3, 160, 960])


In [37]:
out = torchvision.utils.make_grid(images)
print("out-size:", out.shape)

#plt.imshow(out)

out-size: torch.Size([3, 326, 7698])


In [38]:
net = torchvision.models.resnet18(pretrained=True)
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#net = net.cuda() if device else net
net

C:\Users\morit\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\morit\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Scaling


In [109]:
def normalize_img(tensor):
    v_min, v_max = tensor.min(), tensor.max()
    new_min, new_max = 0, 1
    v_p = (tensor - v_min)/(v_max - v_min)*(new_max - new_min) + new_min
    return v_p

batch = next(iter(train_dataloader))
img = batch["rgb"][0][0,:,:,:].double()
img.shape
#print(normalize_img(img))

torch.Size([160, 960, 3])

In [110]:
img2 = torch.randn(160,960,3)

In [106]:
torch.stack([img,img2],dim=0).shape

torch.Size([2, 160, 960, 3])

In [122]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

num_ftrs = net.fc.in_features

net.fc = nn.Sequential(
    nn.Linear(num_ftrs, 4), #nbr of outputs
    nn.Tanh()
)
net.fc = net.fc.cuda() if False else net.fc

In [ ]:
%%time

n_epochs = 1
print_every = 10
valid_loss_min = np.Inf
val_loss = []
train_loss = []
total_step = len(train_dataloader)

for epoch in range(1, n_epochs+1):
    
    running_loss = 0.0
    print(f'Epoch {epoch}\n')
    
    #data: (['idx', 'rgb', 'speed', 'steer', 'throttle', 'brake'])
    for batch_idx, data in enumerate(train_dataloader):

        # further preprocessing
        data_ = normalize_img(data["rgb"].transpose(1,4)[:,:,:,:,0]).float()
        #print(type(data_))
        target_ = torch.stack([data["speed"],data["steer"],data["throttle"],data["brake"]]).float()
        #target_ = {"speed":speed, "steer":steer,"throttle":throttle,"brake":brake}
        
        #data_, target_ = data_.to(device), target_.to(device)
        optimizer.zero_grad()
        
        outputs = net(data_)
        loss = criterion(outputs, target_) # ERROR
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (batch_idx) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))

    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f},')
    batch_loss = 0
    total_t=0
    correct_t=0
    with torch.no_grad():
        net.eval()
        for data_t, target_t in (test_dataloader):
            data_t, target_t = data_t.to(device), target_t.to(device)
            outputs_t = net(data_t)
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
        val_loss.append(batch_loss/len(test_dataloader))
        #network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, \n')

        
        if False:#network_learned:
            valid_loss_min = batch_loss
            torch.save(net.state_dict(), 'resnet.pt')
            print('Improvement-Detected, save-model')
    net.train()

Epoch 1

Epoch [1/1], Step [0/126], Loss: 2.0973
Epoch [1/1], Step [20/126], Loss: 2.2428
Epoch [1/1], Step [40/126], Loss: 1.5484
